In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## 0.Get SAMSUM dataset

In [ ]:
!pip install datasets
# !pip install py7zr
!pip install transformers
!pip install rouge_score
!pip install sentencepiece

     |████████████████████████████████| 325 kB 4.9 MB/s 
     |████████████████████████████████| 77 kB 3.9 MB/s 
     |████████████████████████████████| 136 kB 63.9 MB/s 
     |████████████████████████████████| 212 kB 43.3 MB/s 
     |████████████████████████████████| 1.1 MB 45.8 MB/s 
     |████████████████████████████████| 127 kB 48.8 MB/s 
     |████████████████████████████████| 144 kB 62.0 MB/s 
     |████████████████████████████████| 94 kB 514 kB/s 
     |████████████████████████████████| 271 kB 68.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.0 MB 4.7 MB/s 


In [ ]:
!pip install py7zr

     |████████████████████████████████| 4.3 MB 5.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 357 kB 55.9 MB/s 
     |████████████████████████████████| 131 kB 70.8 MB/s 
     |████████████████████████████████| 2.4 MB 53.8 MB/s 
     |████████████████████████████████| 57 kB 4.9 MB/s 
     |████████████████████████████████| 2.0 MB 56.3 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
  Created wheel for py7zr: filename=py7zr-0.18.5-py3-none-any.whl size=69901 sha256=20a24ca610691922ab5d174bf80f9acfe6615212e5549e08a24faa480d0c9208
  Stored in directory: /root/.cache/pip/wheels/c8/f6/f8/9a860b95b853b2a891d478d2d2afde2fc0460514a7015d15d5
Successfully built py7zr


In [ ]:
from datasets import load_dataset
dataset = load_dataset("samsum")

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
dataset["train"][0]

{'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'id': '13818513',
 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.'}

In [ ]:
dataset["train"][0]["dialogue"]

"Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"

In [ ]:
dataset["train"][0]["summary"]

'Amanda baked cookies and will bring Jerry some tomorrow.'

In [ ]:
import pandas as pd
train_dataset = pd.DataFrame(dataset["train"])
val_dataset = pd.DataFrame(dataset["validation"])
test_dataset = pd.DataFrame(dataset["test"])

## 1. Baseline model

In [ ]:
# !pip install "sagemaker>=2.48.0" "transformers==4.6.1" "datasets[s3]==1.6.2" --upgrade

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")

In [ ]:
def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

In [ ]:
test_dataset["summary"][0]

"Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry."

In [ ]:
calc_rouge_scores(test_dataset["summary"][0], test_dataset["summary"][0])

{'rouge1': 79.5, 'rouge2': 0.0, 'rougeL': 79.5, 'rougeLsum': 79.5}

In [ ]:
ref_summaries = list(test_dataset["summary"])

In [ ]:
ref_summaries[:5]

["Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.",
 'Eric and Rob are going to watch a stand-up on youtube.',
 "Lenny can't decide which trousers to buy. Bob advised Lenny on that topic. Lenny goes with Bob's advice to pick the trousers that are of best quality.",
 'Emma will be home soon and she will let Will know.',
 "Jane is in Warsaw. Ollie and Jane has a party. Jane lost her calendar. They will get a lunch this week on Friday. Ollie accidentally called Jane and talked about whisky. Jane cancels lunch. They'll meet for a tea at 6 pm."]

In [ ]:
import re
for i in range (10):
    candidate_summaries = list(test_dataset["dialogue"].apply(lambda x: ' '.join(re.split(r'(?<=[.:;])\s', x)[:i+1])))
    print(f"First {i+1} senctences: Scores {calc_rouge_scores(candidate_summaries, ref_summaries)}")

First 1 senctences: Scores {'rouge1': 10.1, 'rouge2': 0.0, 'rougeL': 10.1, 'rougeLsum': 10.1}
First 2 senctences: Scores {'rouge1': 26.5, 'rouge2': 6.6, 'rougeL': 21.9, 'rougeLsum': 24.1}
First 3 senctences: Scores {'rouge1': 29.3, 'rouge2': 7.4, 'rougeL': 23.4, 'rougeLsum': 26.6}
First 4 senctences: Scores {'rouge1': 30.7, 'rouge2': 8.0, 'rougeL': 24.0, 'rougeLsum': 27.5}
First 5 senctences: Scores {'rouge1': 31.6, 'rouge2': 8.7, 'rougeL': 24.3, 'rougeLsum': 28.3}
First 6 senctences: Scores {'rouge1': 32.0, 'rouge2': 9.2, 'rougeL': 24.4, 'rougeLsum': 28.6}
First 7 senctences: Scores {'rouge1': 32.0, 'rouge2': 9.5, 'rougeL': 24.3, 'rougeLsum': 28.6}
First 8 senctences: Scores {'rouge1': 31.7, 'rouge2': 9.7, 'rougeL': 24.1, 'rougeLsum': 28.3}
First 9 senctences: Scores {'rouge1': 31.3, 'rouge2': 9.7, 'rougeL': 23.6, 'rougeLsum': 27.9}
First 10 senctences: Scores {'rouge1': 31.0, 'rouge2': 9.8, 'rougeL': 23.3, 'rougeLsum': 27.7}


In [ ]:
original_test = list(test_dataset["dialogue"].head())
original_test[0]

"Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye"

In [ ]:
candidate_summaries_demo = list(test_dataset["dialogue"].apply(lambda x: ' '.join(re.split(r'(?<=[.:;])\s', x)[:6])))
candidate_summaries_demo[:5]

["Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it. Amanda:",
 "Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric: I know! And shows how Americans see Russian ;)\r\nRob: And it's really funny!\r\nEric: I know! I especially like the train part!\r\nRob:",
 "Lenny: Babe, can you help me with something?\r\nBob: Sure, what's up?\r\nLenny: Which one should I pick?\r\nBob: Send me photos\r\nLenny:  <file_photo>\r\nLenny:",
 "Will: hey babe, what do you want for dinner tonight?\r\nEmma:  gah, don't even worry about it tonight\r\nWill: what do you mean? everything ok?\r\nEmma: not really, but it's ok, don't worry about cooking though, I'm not hungry\r\nWill: Well what time will you be home?\r\nEmma:",
 'Ollie: Hi , are you in Warsaw\r\nJane: yes, just back! Btw are you free for diner the 19th?\r\nOllie: nope!\r\nJane: and the  18th?\r\nOllie: nope, we have this party and you must be there, remember?\r\nJane:']

In [ ]:
ref_summaries[:5]

["Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.",
 'Eric and Rob are going to watch a stand-up on youtube.',
 "Lenny can't decide which trousers to buy. Bob advised Lenny on that topic. Lenny goes with Bob's advice to pick the trousers that are of best quality.",
 'Emma will be home soon and she will let Will know.',
 "Jane is in Warsaw. Ollie and Jane has a party. Jane lost her calendar. They will get a lunch this week on Friday. Ollie accidentally called Jane and talked about whisky. Jane cancels lunch. They'll meet for a tea at 6 pm."]

## 2. Pretrained Models (pegasus & bart) zero-shot learning (ZSL)

In [ ]:
from transformers import pipeline
summarizer_pegasus = pipeline("summarization", model='google/pegasus-xsum')
summarizer_bart = pipeline("summarization", model='facebook/bart-large-cnn')
summarizer_bart_samsum = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")
summarizer_pegasus_samsum = pipeline("summarization", model="jpcorb20/pegasus-large-reddit_tifu-samsum-512")

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

In [ ]:
ref_summaries = list(test_dataset["summary"])
texts = list(test_dataset["dialogue"])

In [ ]:
ref_summaries[0]

"Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry."

In [ ]:
texts[0]

"Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye"

In [ ]:
summarizer_pegasus(texts[0])[0]["summary_text"]

"Amanda: Hey Hannah, do you have Betty's number?"

In [ ]:
summarizer_bart(texts[0])

Your max_length is set to 142, but you input_length is only 139. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=69)


[{'summary_text': 'Hannah asks Amanda for Betty\'s number. Amanda can\'t find it. Hannah asks Larry. Amanda asks Larry to text him. Hannah says she\'ll text him back. Hannah calls it a day and says she\'s going to go home. Hannah: "Bye bye"'}]

In [ ]:
summarizer_pegasus_samsum(texts[0])

Your max_length is set to 128, but you input_length is only 122. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


[{'summary_text': "Amanda Amanda Amanda Amanda Amanda. Larry Larry. Larry called Betty's last time they were at the park together at the park together. Amanda will text Larry."}]

In [ ]:
summarizer_bart_samsum(texts[0])

Your max_length is set to 142, but you input_length is only 139. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=69)


[{'summary_text': "Larry called Betty the last time Hannah and Amanda were at the park together. Hannah doesn't know Larry very well. Amanda suggests Hannah to text Larry to ask him for Betty's number. Hannah agrees to text him. Hannah and Amanda are going to talk later."}]

In [ ]:
len(texts)

2793

In [ ]:
candidate_summaries_pegasus = []
candidate_summaries_bart = []
candidate_summaries_pegasus_samsum = []
candidate_summaries_bart_samsum = []
for text_id, text in enumerate(texts[351]):
  # candidate_bart = summarizer_bart(text, min_length=10, max_length=30)
  # candidate_pegasus = summarizer_pegasus(text, min_length=10, max_length=30)
  # candidate_bart_samsum = summarizer_bart_samsum(text, min_length=10, max_length=30)
  candidate_pegasus_samsum = summarizer_pegasus_samsum(text, min_length=10, max_length=30)
  print(text_id)

  # candidate_summaries_bart.append(candidate_bart[0]["summary_text"])
  # candidate_summaries_pegasus.append(candidate_pegasus[0]["summary_text"])
  # candidate_summaries_bart_samsum.append(candidate_bart_samsum[0]["summary_text"])
  candidate_summaries_pegasus_samsum.append(candidate_pegasus_samsum[0]["summary_text"])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


Your max_length is set to 30, but you input_length is only 29. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)


32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105


Your max_length is set to 30, but you input_length is only 27. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)


106
107
108
109
110
111
112
113
114
115
116
117
118
119
120


...

Your max_length is set to 30, but you input_length is only 21. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)


347
348


Token indices sequence length is longer than the specified maximum sequence length for this model (691 > 512). Running this sequence through the model will result in indexing errors


349


IndexError: ignored

In [ ]:
file = open("./zero-shot-summaries-bart_part4.txt", "w")
for s in candidate_summaries_bart:
    file.write(s + "\n")
file.close()

In [ ]:
file = open("./zero-shot-summaries-pegasus_part4.txt", "w")
for s in candidate_summaries_pegasus:
    file.write(s + "\n")
file.close()

In [ ]:
file = open("./zero-shot-summaries-bart-samsum_part4.txt", "w")
for s in candidate_summaries_bart_samsum:
    file.write(s + "\n")
file.close()

In [ ]:
file = open("./zero-shot-summaries-pegasus-samsum_part1.txt", "w")
for s in candidate_summaries_pegasus_samsum:
    file.write(s + "\n")
file.close()

## 3. Wyniki - metryka ROUGE

In [ ]:
import numpy as np
bart = np.loadtxt("/content/zero-shot-summaries-bart.txt", delimiter="\n", dtype=str)
bart_samsum = np.loadtxt("/content/zero-shot-summaries-bart-samsum.txt", delimiter="\n", dtype=str)
pegasus = np.loadtxt("/content/zero-shot-summaries-pegasus.txt", delimiter="\n", dtype=str)
pegasus_samsum = np.loadtxt("/content/zero-shot-summaries-pegasus-samsum_part1.txt", delimiter="\n", dtype=str)

In [ ]:
calc_rouge_scores(bart,ref_summaries[:350])

{'rouge1': 33.9, 'rouge2': 11.0, 'rougeL': 27.0, 'rougeLsum': 26.9}

In [ ]:
calc_rouge_scores(bart_samsum,ref_summaries[:350])

{'rouge1': 52.6, 'rouge2': 28.5, 'rougeL': 43.9, 'rougeLsum': 43.9}

In [ ]:
calc_rouge_scores(pegasus,ref_summaries[:350])

{'rouge1': 23.5, 'rouge2': 3.9, 'rougeL': 18.2, 'rougeLsum': 18.2}

In [ ]:
calc_rouge_scores(pegasus_samsum,ref_summaries[:350])

{'rouge1': 19.4, 'rouge2': 6.9, 'rougeL': 17.0, 'rougeLsum': 16.9}

In [ ]:
ref_summaries[:30]

["Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.",
 'Eric and Rob are going to watch a stand-up on youtube.',
 "Lenny can't decide which trousers to buy. Bob advised Lenny on that topic. Lenny goes with Bob's advice to pick the trousers that are of best quality.",
 'Emma will be home soon and she will let Will know.',
 "Jane is in Warsaw. Ollie and Jane has a party. Jane lost her calendar. They will get a lunch this week on Friday. Ollie accidentally called Jane and talked about whisky. Jane cancels lunch. They'll meet for a tea at 6 pm.",
 "Hilary has the keys to the apartment. Benjamin wants to get them and go take a nap. Hilary is having lunch with some French people at La Cantina. Hilary is meeting them at the entrance to the conference hall at 2 pm. Benjamin and Elliot might join them. They're meeting for the drinks in the evening.",
 'Payton provides Max with websites selling clothes. Payton likes browsing and trying on the clothes but not nec